# Алгоритмы интеллектуальной обработки больших объемов данных
## Семинар, учимся настраивать параметры бустинга


Разберем, например, xgboost:
https://xgboost.readthedocs.io/en/latest/parameter.html

https://xgboost.readthedocs.io/en/latest/python/python_api.html

Параметры можно разделить на группы. 



1) Общие параметры алгоритма, например, тип бустинга

2) Параметры обучения бустинга, например число итераций, learning rate

3) Параметры построения деревьев, например, глубина, минимальное число элементов в листе

4) Параметры стохастики, subsample, colsample

5) Регуляриазция, например, lambda


ПРАВИЛЬНОГО алгоритма для тюнинга нет, вот один из вариантов, которые часто используют на практике.

1) Возьмите дефолтное параметры. Поставьте нужные loss и eval функции.  Измените максимальное  число деревьев, чтобы было не долго делать эксперименты

2) Подберите для него alpha, чтобы алгоритм не переобучался

3) Подберите параметры сложности дерева, например,  глубину, мин число объектов в листах, мин значение для расщепления и тд

4) Подберите паарметры стохастики

5) Подберите регуляризацию

6) Уменьшите leraning rate и обучите максимальное число деревьев, чтобы не переобучалось

Давайте настроим параметры бустинга на нашем конкурсе!

In [1]:
doc_to_title = {}
with open('docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))


28026


In [2]:
import pandas as pd
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

In [358]:
len(traingroups_titledata)

129

In [359]:
import numpy as np
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:25]    )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11690, 25) (11690,) (11690,)


Подберите размер батча для обучения. Линейная модель не должна учиться дольше нескольких минут. 

Не забывайте использовать скейлер!

In [6]:
!pip install lightgbm
!pip install xgboost

     |████████████████████████████████| 1.2 MB 1.4 MB/s eta 0:00:01


In [12]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
clf = xgb.XGBClassifier()
clf = lgb.LGBMClassifier()

In [360]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
my_sk = StandardScaler()
X_train_sk = my_sk.fit_transform(X_train)

1) Возьмите дефолтное параметры. Поставьте нужные loss и eval функции. Измените максимальное число деревьев, чтобы было не долго делать эксперименты

In [15]:
clf = xgb.XGBClassifier(booster='gbtree', verbosity=2)

In [17]:
%timeit clf.fit(X_train_sk, y_train)

[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 82 extra nodes, 0 pruned nodes, max_depth=6
[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 82 extra nodes, 0 pruned nodes, max_depth=6
[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 92 extra nodes, 0 pruned nodes, max_depth=6
[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 0 pruned nodes, max_depth=6
[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=6
[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 72 extra nodes, 0 pruned nodes, max_depth=6
[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 0 pruned nodes, max_depth=6

[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 76 extra nodes, 0 pruned nodes, max_depth=6
[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=6
[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=6
[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=6
[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=6
[21:13:04] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 16 extra nodes, 0 pruned nodes, max_depth=6

[21:13:05] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=6
[21:13:05] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=6
[21:13:05] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=6
[21:13:05] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=6
[21:13:05] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 64 extra nodes, 0 pruned nodes, max_depth=6
[21:13:05] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=6
[21:13:05] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=6

[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=6
[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 72 extra nodes, 0 pruned nodes, max_depth=6
[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 0 pruned nodes, max_depth=6
[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 74 extra nodes, 0 pruned nodes, max_depth=6
[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 76 extra nodes, 0 pruned nodes, max_depth=6
[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 76 extra nodes, 0 pruned nodes, max_depth=6
[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 64 extra nodes, 0 pruned nodes, max_depth=6

[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=6
[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=6
[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=6
[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=6
[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=6
[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 76 extra nodes, 0 pruned nodes, max_depth=6
[21:13:06] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=6

...

[21:13:07] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=6
[21:13:07] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=6
[21:13:07] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=6
[21:13:07] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=6
[21:13:07] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 70 extra nodes, 0 pruned nodes, max_depth=6
[21:13:07] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=6
[21:13:07] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6

[21:13:08] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 74 extra nodes, 0 pruned nodes, max_depth=6
[21:13:08] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 98 extra nodes, 0 pruned nodes, max_depth=6
[21:13:08] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=6
[21:13:08] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 0 pruned nodes, max_depth=6
[21:13:08] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 82 extra nodes, 0 pruned nodes, max_depth=6
[21:13:08] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=6
[21:13:08] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=6

[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=6
[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 78 extra nodes, 0 pruned nodes, max_depth=6
[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=6
[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=6
[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 0 pruned nodes, max_depth=6
[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 74 extra nodes, 0 pruned nodes, max_depth=6
[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 82 extra nodes, 0 pruned nodes, max_depth=6

[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6
[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6
[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6
[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6
[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=6
[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 58 extra nodes, 0 pruned nodes, max_depth=6
[21:13:09] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 76 extra nodes, 0 pruned nodes, max_depth=6

[21:13:10] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 16 extra nodes, 0 pruned nodes, max_depth=6
[21:13:10] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=6
[21:13:10] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=6
[21:13:10] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=6
[21:13:10] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=6
[21:13:10] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 70 extra nodes, 0 pruned nodes, max_depth=6
[21:13:10] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=6

In [18]:
from sklearn.model_selection import StratifiedKFold

In [51]:
cv = StratifiedKFold(n_splits=5, shuffle=True)
for train_ind, test_ind in cv.split(X_train_sk, y_train):
    clf = xgb.XGBClassifier(booster='gbtree', verbosity=2)
    clf.fit(X_train_sk[train_ind], y_train[train_ind])
    y_pred = clf.predict(X_train_sk[test_ind])
    print(f1_score(y_train[test_ind], y_pred ))

[00:34:27] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 0 pruned nodes, max_depth=6
[00:34:27] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 0 pruned nodes, max_depth=6
[00:34:27] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 72 extra nodes, 0 pruned nodes, max_depth=6
[00:34:27] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 82 extra nodes, 0 pruned nodes, max_depth=6
[00:34:27] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 86 extra nodes, 0 pruned nodes, max_depth=6
[00:34:27] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=6
[00:34:27] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 0 pruned nodes, max_depth=6

[00:34:27] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6
[00:34:27] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=6
[00:34:27] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=6
[00:34:28] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=6
[00:34:28] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=6
[00:34:28] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=6
[00:34:28] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=6

[00:34:28] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 16 extra nodes, 0 pruned nodes, max_depth=6
[00:34:28] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 0 pruned nodes, max_depth=6
[00:34:28] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=6
[00:34:28] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 0 pruned nodes, max_depth=6
[00:34:28] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=6
[00:34:28] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[00:34:28] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=6

[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 0 pruned nodes, max_depth=6
[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=6
[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=6
[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=6
[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=6
[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 72 extra nodes, 0 pruned nodes, max_depth=6
[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=6

[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 0 pruned nodes, max_depth=6
[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6
[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=6
[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=6
[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=6
[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6
[00:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6

[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=6
[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=6
[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=6
[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=6
[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=6
[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=6
[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=6

[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=6
[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=6
[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=6
[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=6
[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=6
[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=6
[00:34:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=6

In [49]:
def for_time():
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=150)
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        print(f1_score(y_train[test_ind], y_pred ))

In [50]:
cv = StratifiedKFold(n_splits=5, shuffle=True)
%timeit for_time()

0.5596491228070175
0.6132879045996592
0.576889661164205
0.5676625659050967
0.6053097345132744
0.5955555555555556
0.5796847635726795
0.6012269938650306
0.5728987993138938
0.5796847635726795
0.5612153708668455
0.5862676056338029
0.5893635571054926
0.6025316455696202
0.5996563573883162
0.5819456617002629
0.552158273381295
0.6181818181818182
0.5719329214474845
0.5664335664335665
0.5803649000868809
0.576271186440678
0.5701754385964912
0.5911840968020744
0.5761821366024519
0.5845620121422376
0.5726569217540842
0.5817223198594026
0.5619982158786797
0.6007005253940455
0.5874001774622893
0.5709251101321586
0.5809859154929577
0.5936151855047455
0.5784832451499118
0.5746724890829693
0.5773552290406223
0.5933682373472948
0.6069906223358909
0.5685920577617328
4.42 s ± 263 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


2) Подберите для него alpha, чтобы алгоритм не переобучался

In [53]:
etas = [0, 0.01, 0.1, 0.3, 0.5, 0.8, 1]
cv = StratifiedKFold(n_splits=5, shuffle=True)
for eta in etas:
    print(eta)
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=150, eta=eta)
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        print(f1_score(y_train[test_ind], y_pred ))

0
0.0
0.0
0.0
0.0
0.0
0.01
0.5859982713915297
0.5517241379310345
0.5784832451499118
0.5629370629370629
0.6053310404127257
0.1
0.5681818181818182
0.6088464874241111
0.5894736842105263
0.5969615728328865
0.562390158172232
0.3
0.5519650655021834
0.6108108108108108
0.5690093141405589
0.5783348254252463
0.603284356093345
0.5
0.5579322638146167
0.5794066317626528
0.5942658557775847
0.5866900175131349
0.5982905982905984
0.8
0.5669835782195333
0.5681618293755497
0.5814977973568282
0.5602836879432624
0.5857385398981324
1
0.5751295336787565
0.556808326105811
0.5839793281653747
0.5709515859766278
0.5726643598615916


In [54]:
etas = [0.1, 0.15, 0.2, 0.25, 0.3]
cv = StratifiedKFold(n_splits=5, shuffle=True)
for eta in etas:
    print(eta)
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=150, eta=eta)
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        print(f1_score(y_train[test_ind], y_pred ))

0.1
0.5855379188712523
0.5874246339362618
0.601328903654485
0.5960502692998205
0.5811051693404634
0.15
0.5676156583629893
0.587012987012987
0.6284751474304971
0.5952589991220368
0.5794556628621599
0.2
0.5864527629233511
0.5918189730200174
0.596551724137931
0.5748917748917749
0.5922818791946308
0.25
0.5770212765957446
0.6007067137809188
0.5861767279090113
0.5754141238012206
0.5668161434977579
0.3
0.5873287671232876
0.5761821366024518
0.6000000000000001
0.6062717770034843
0.5640569395017795


eta = 0.1

In [57]:
alphas = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
cv = StratifiedKFold(n_splits=5, shuffle=True)
for alpha in alphas:
    print(alpha)
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=150, eta=0.1, alpha=alpha)
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        print(f1_score(y_train[test_ind], y_pred ))

0
0.5789473684210527
0.5812720848056537
0.5738980121002593
0.5871725383920505
0.6112026359143328
0.0001
0.5874125874125875
0.5886524822695035
0.5804749340369393
0.5605536332179931
0.5852372583479789
0.001
0.6107784431137725
0.5609973285841495
0.5724572457245725
0.5771578029642547
0.5810344827586207
0.01
0.5880281690140845
0.5787610619469026
0.5706713780918728
0.5800711743772242
0.607659574468085
0.1
0.5779735682819382
0.5738980121002593
0.5934819897084048
0.5752212389380532
0.5893635571054926
1
0.5754385964912281
0.5884383088869715
0.5945470536499561
0.5880341880341879
0.5771929824561403
10
0.5900783289817232
0.5756780402449694
0.5791304347826087
0.5756521739130434
0.5609318996415771
100
0.5550500454959054
0.571945701357466
0.546617915904936
0.535288725939505
0.5761821366024519
1000
0.0
0.0
0.0
0.0
0.0


alpha = 0.01
проверил также параметр lambda, он не работает и игнорируется

In [58]:
gammas = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
cv = StratifiedKFold(n_splits=5, shuffle=True)
for gamma in gammas:
    print(gamma)
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=150, eta=0.1, alpha=0.01, gamma=gamma)
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        print(f1_score(y_train[test_ind], y_pred ))

0
0.5930434782608696
0.5709188224799286
0.5741878841088675
0.5923344947735191
0.6068376068376068
0.0001
0.5676625659050967
0.5711711711711712
0.5792253521126761
0.6109660574412533
0.6037099494097807
0.001
0.5567375886524822
0.587521663778163
0.6090675791274593
0.5893805309734513
0.5709312445604873
0.01
0.5817555938037866
0.6197424892703862
0.6010273972602739
0.5808757819481679
0.5585106382978724
0.1
0.6043668122270741
0.5817223198594026
0.5648312611012434
0.5829750644883921
0.5945945945945946
1
0.5980985306828004
0.5765611633875106
0.5601436265709157
0.5934664246823956
0.5942760942760943
10
0.5815861440291705
0.5550581915846016
0.5547826086956521
0.5466297322253001
0.5648148148148149
100
0.5674931129476584
0.532347504621072
0.5514705882352942
0.5307971014492754
0.5502347417840376
1000
0.0
0.0
0.0
0.0
0.0


3) Подберите параметры сложности дерева, например, глубину, мин число объектов в листах, мин значение для расщепления и тд

In [60]:
depths = [3, 6, 10, 12, 20, 50]
cv = StratifiedKFold(n_splits=5, shuffle=True)
for d in depths:
    print(d)
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=150, eta=0.1, alpha=0.01, max_depth=d)
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        print(f1_score(y_train[test_ind], y_pred ))

3
0.5796344647519581
0.5899912203687444
0.572419774501301
0.5658362989323843
0.6140503035559409
6
0.5971731448763251
0.5834061135371179
0.5688405797101448
0.5610389610389611
0.5936698032506416
10
0.5940766550522648
0.575152041702867
0.5746924428822495
0.5998271391529819
0.5842696629213483
12
0.5772843723313408
0.5833333333333334
0.5853227232537577
0.5831202046035806
0.5888501742160279
20
0.5598621877691645
0.5716814159292035
0.5886344359626804
0.5924050632911392
0.5787671232876712
50
0.5860621326616289
0.5818181818181818
0.5665236051502146
0.5643648763853367
0.5547945205479452


In [62]:
depths = [i for i in range(2, 13)]
cv = StratifiedKFold(n_splits=5, shuffle=True)
for d in depths:
    print(d)
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=150, eta=0.1, alpha=0.01, max_depth=d)
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        print(f1_score(y_train[test_ind], y_pred ))

2
0.6111587982832617
0.5607305936073059
0.5683836589698046
0.5758889852558543
0.5777400169923534
3
0.5928449744463373
0.5763589301121657
0.6074201898188094
0.5481481481481482
0.580875781948168
4
0.5916230366492146
0.5957081545064378
0.564240790655885
0.5714285714285714
0.5755395683453237
5
0.6106346483704975
0.5701519213583556
0.5646853146853147
0.577391304347826
0.592328278322926
6
0.5970664365832614
0.5813751087902524
0.5827338129496403
0.5837742504409171
0.583904109589041
7
0.5879310344827585
0.5764809902740937
0.5671378091872791
0.5954063604240283
0.591747146619842
8
0.5824742268041238
0.6048951048951049
0.5756780402449694
0.5850694444444444
0.578806767586821
9
0.5704467353951891
0.57243195785777
0.6037399821905609
0.5765004226542688
0.5994694960212201
10
0.6068222621184919
0.5612788632326821
0.5634782608695652
0.5793581960104077
0.5959933222036728
11
0.5734639358860196
0.6021867115222876
0.5894378194207837
0.6019080659150042
0.5681618293755497
12
0.5491949910554562
0.5728727885425

max_depth = 6 / 8

In [76]:
min_child_weights = [1, 2, 5, 10, 100, 1000]
cv = StratifiedKFold(n_splits=5, shuffle=True)
for elem in min_child_weights:
    print(elem)
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=150, eta=0.1, alpha=0.01, max_depth=6, min_child_weight=elem)
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        print(f1_score(y_train[test_ind], y_pred ))

1
0.6024305555555556
0.5734388742304309
0.5733905579399142
0.5648312611012434
0.6090675791274595
2
0.5736981465136805
0.6003552397868561
0.5832614322691976
0.5947368421052631
0.5689201053555751
5
0.5935030728709395
0.5974248927038627
0.575152041702867
0.593886462882096
0.5631067961165048
10
0.5884383088869715
0.5963382737576286
0.5838401390095569
0.5709219858156029
0.5729257641921398
100
0.5714285714285714
0.5998315080033698
0.5892700087950747
0.5689045936395759
0.5363636363636364
1000
0.0
0.0
0.0
0.0
0.0


In [77]:
min_child_weights = [i for i in range(11)]
cv = StratifiedKFold(n_splits=5, shuffle=True)
for elem in min_child_weights:
    print(elem)
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=150, eta=0.1, alpha=0.01, max_depth=6, min_child_weight=elem)
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        print(f1_score(y_train[test_ind], y_pred ))

0
0.5752212389380532
0.5987920621225195
0.5806451612903226
0.5856255545696539
0.5804749340369393
1
0.5737265415549598
0.5938566552901023
0.6035805626598465
0.5873153779322329
0.5962399283795882
2
0.5784399649430323
0.5797356828193833
0.601195559350982
0.5933682373472948
0.5481874447391688
3
0.6013986013986015
0.5851979345955249
0.5759162303664922
0.5591985428051002
0.5872042068361086
4
0.5756780402449694
0.5784399649430323
0.5766233766233766
0.5763016157989228
0.5797872340425532
5
0.5671111111111111
0.5885416666666666
0.5562556255625563
0.5776173285198556
0.6027164685908319
6
0.5741379310344827
0.588546255506608
0.576271186440678
0.5577758470894875
0.6038732394366197
7
0.5914529914529915
0.5823686553873553
0.5953408110440034
0.5772646536412078
0.5803108808290155
8
0.5866900175131349
0.5795555555555556
0.5936651583710406
0.5741216795201372
0.5699208443271767
9
0.5830388692579506
0.5825242718446602
0.6100254885301614
0.578806767586821
0.5578947368421053
10
0.5854092526690391
0.5709281961

min_child_weight = 1 / 7

In [80]:
subsamples = [0.1, 0.2, 0.275, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
cv = StratifiedKFold(n_splits=5, shuffle=True)
for elem in subsamples:
    print(elem)
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=150, eta=0.1, alpha=0.01, max_depth=6, subsample=elem)
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        print(f1_score(y_train[test_ind], y_pred ))

0.1
0.5824742268041238
0.581151832460733
0.5584642233856895
0.5658012533572068
0.5912596401028277
0.2
0.5593073593073593
0.5887372013651876
0.5815358067299397
0.5824561403508771
0.6060100166944908
0.275
0.5831202046035806
0.588957055214724
0.5649520488230165
0.5825932504440497
0.5897658282740676
0.3
0.5536823425022183
0.5686789151356081
0.5919589392643284
0.6078925272879933
0.5863874345549739
0.4
0.5981620718462823
0.5807017543859649
0.5749128919860627
0.6020583190394512
0.6024518388791593
0.5
0.5848563968668408
0.6042918454935623
0.573943661971831
0.581151832460733
0.5853658536585366
0.6
0.5719207579672696
0.6026200873362445
0.5905579399141632
0.577301161751564
0.5804195804195804
0.7
0.5931623931623932
0.5800865800865801
0.5921397379912663
0.5818815331010453
0.611864406779661
0.8
0.5929734361610968
0.5780445969125214
0.5969615728328865
0.5756266205704409
0.5752212389380531
0.9
0.586597040905135
0.5833333333333334
0.5803108808290155
0.5919589392643284
0.598939929328622


subsample = 0.5 / 0.7

In [83]:
for train_ind, test_ind in cv.split(X_train_sk, y_train):
    clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=200, eta=0.1, alpha=0.01, max_depth=6)
    clf.fit(X_train_sk[train_ind], y_train[train_ind])
    y_pred = clf.predict(X_train_sk[test_ind])
    print(f1_score(y_train[test_ind], y_pred ))

0.5822339489885665
0.5864527629233511
0.5894554883318929
0.5934819897084048
0.584426946631671


In [86]:
eval_metrics = ['rmse', 'rmsle', 'mae', 'mphe', 'logloss', 'auc']
cv = StratifiedKFold(n_splits=5, shuffle=True)
for elem in eval_metrics:
    print(elem)
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=150, eta=0.1, alpha=0.01, max_depth=6, eval_metric=elem)
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        print(f1_score(y_train[test_ind], y_pred ))

rmse
0.548187444739169
0.5863247863247862
0.5987488829311884
0.5998256320836967
0.5818505338078293
rmsle
0.6159052453468696
0.5559633027522936
0.5908683974932856
0.5924006908462867
0.5478723404255319
mae
0.5733788395904437
0.5949367088607594
0.5901926444833625
0.6080273270708796
0.5639229422066551
mphe
0.5893805309734513
0.5869191049913941
0.5914260717410322
0.583044982698962
0.5818815331010453
logloss
0.5898123324396782
0.5747330960854092
0.601195559350982
0.5835453774385073
0.5824561403508772
auc
0.5766233766233766
0.5778175313059034
0.6022433132010354
0.5799648506151143
0.5843694493783305


mphe, logloss, auc 

In [89]:
base_scores = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
cv = StratifiedKFold(n_splits=5, shuffle=True)
for elem in base_scores:
    print(elem)
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=150, eta=0.1, alpha=0.01, max_depth=6, eval_metric='auc', base_score=elem)
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        print(f1_score(y_train[test_ind], y_pred ))

0.1
0.5642857142857143
0.5921985815602837
0.5916230366492146
0.5774410774410774
0.5991111111111111
0.2
0.5737265415549598
0.5655296229802513
0.6096069868995634
0.5851528384279475
0.5896147403685092
0.3
0.5899280575539569
0.5653710247349822
0.5953237410071943
0.5939914163090129
0.5777777777777778
0.4
0.5942658557775847
0.6071741032370954
0.5913338997451147
0.5940246045694201
0.5766871165644171
0.5
0.5886524822695035
0.5691629955947136
0.6226415094339622
0.5664939550949913
0.57117903930131
0.6
0.5869759143621766
0.5979020979020979
0.5837742504409171
0.5694323144104803
0.5911840968020743
0.7
0.6034782608695652
0.5709251101321586
0.5785714285714286
0.6013513513513514
0.5746724890829694


In [91]:
for train_ind, test_ind in cv.split(X_train_sk, y_train):
    clf = xgb.XGBClassifier(booster='gbtree', verbosity=1, n_estimators=2000, eta=0.1, alpha=0.01, max_depth=6, eval_metric='auc')
    clf.fit(X_train_sk[train_ind], y_train[train_ind])
    y_pred = clf.predict(X_train_sk[test_ind])
    print(f1_score(y_train[test_ind], y_pred ))

0.5616797900262467
0.5877192982456141
0.5665773011617515
0.5907172995780591
0.5471014492753623


Здесь я решил, что что-то делаю не так и начал все заново, написал вспомогательную функцию, чтобы было меньше дублирования кода)

In [98]:
from statistics import mean

In [376]:
def my_score(params):
    cv = StratifiedKFold(n_splits=5, shuffle=True)
    clf = xgb.XGBClassifier(**params)
    m = []
    for train_ind, test_ind in cv.split(X_train_sk, y_train):
        clf.fit(X_train_sk[train_ind], y_train[train_ind])
        y_pred = clf.predict(X_train_sk[test_ind])
        #print(f1_score(y_train[test_ind], y_pred ))
        m.append(f1_score(y_train[test_ind], y_pred ))
    return mean(m)

In [200]:
my_score({'n_estimators':10})

0.5960810826848206

In [203]:
for max_depth in range(2, 13):
    print(max_depth)
    print(my_score({'n_estimators':10, 'max_depth':max_depth}))

2
0.5550941212940435
3
0.5687404159007401
4
0.5817859668063383
5
0.5882228819544931
6
0.5877706305767442
7
0.5902739328605725
8
0.5952547126829414
9
0.5923463206335136
10
0.597956496608666
11
0.5903580979995446
12
0.5904034700980745


max_depth = 10

In [210]:
for min_child_weight in range(1, 10):
    print(min_child_weight)
    print(my_score({'n_estimators':10, 'max_depth':10, 'min_child_weight':min_child_weight}))

1
0.5874380622400633
2
0.5975824211757895
3
0.5992088913468343
4
0.5997412914060186
5
0.6000325678951774
6
0.593038284624848
7
0.5923424151093065
8
0.5999004730445825
9
0.5966782762065058


min_child_weight = 5

In [113]:
import numpy as np

In [214]:
for subsample in np.arange(0.5, 1, 0.05):
    print(subsample)
    print(my_score({'n_estimators':10, 'max_depth':10, 'min_child_weight':5, 'subsample':subsample}))

0.5
0.5893317659961546
0.55
0.597407683094871
0.6000000000000001
0.5940078758009004
0.6500000000000001
0.5915911615877315
0.7000000000000002
0.5899300054692425
0.7500000000000002
0.58796698720878
0.8000000000000003
0.5919251536594774
0.8500000000000003
0.5901353029535923
0.9000000000000004
0.5951609318159541
0.9500000000000004
0.6007013476448712


subsample = 0.95

In [217]:
for eval_metric in ['rmse', 'rmsle', 'mae', 'mphe', 'logloss', 'auc']:
    print(eval_metric)
    print(my_score({'n_estimators':10, 'max_depth':10, 'min_child_weight':5, 'subsample':0.95, 'eval_metric':eval_metric}))

rmse
0.5920909568312319
rmsle
0.5910616223283554
mae
0.5955185805321421
mphe
0.5908201118379329
logloss
0.5930141614571545
auc
0.600278885089801


eval_metric = auc

In [260]:
print(my_score({'n_estimators':30, 'max_depth':10, 'min_child_weight':5, 'subsample':0.95, 'eval_metric':'auc'}))

0.594213119245185


In [261]:
save = {'n_estimators':10, 'max_depth':10, 'min_child_weight':5, 'subsample':0.95, 'eval_metric':'auc'}

In [267]:
for gamma in np.arange(0.5, 1, 0.05):
    print(gamma)
    print(my_score({**save, 'gamma':gamma}))

0.5
0.5913654972397524
0.55
0.5917718210534956
0.6000000000000001
0.5993068382523726
0.6500000000000001
0.600074434914092
0.7000000000000002
0.5926731863308878
0.7500000000000002
0.5945583021280771
0.8000000000000003
0.5977106944179909
0.8500000000000003
0.5910019817053134
0.9000000000000004
0.5982090045800735
0.9500000000000004
0.5958832034375385


gamma = 0.65

In [270]:
for colsample_bytree in np.arange(0.5, 1.01, 0.05):
    print(colsample_bytree)
    print(my_score({**save, 'gamma':0.65, 'colsample_bytree':colsample_bytree}))

0.5
0.5925327704741786
0.55
0.5881909397305564
0.6000000000000001
0.5917340288006097
0.6500000000000001
0.5881261283687554
0.7000000000000002
0.5904140580646825
0.7500000000000002
0.5889581716086258
0.8000000000000003
0.5944470051979905
0.8500000000000003
0.5950786059871058
0.9000000000000004
0.5997387444217606
0.9500000000000004
0.5923984836098197
1.0000000000000004
0.598331204282895


colsample_bytree = 0.9

In [274]:
for eta in np.arange(0.025, 0.5, 0.025):
    print(eta)
    print(my_score({**save, 'gamma':0.65, 'colsample_bytree':0.9, 'eta':eta}))

0.025
0.5899045548288093
0.05
0.5974661595802536
0.07500000000000001
0.5945804588158269
0.1
0.5896639166837225
0.125
0.5904775211881985
0.15
0.5917287685210245
0.17500000000000002
0.5859935391114998
0.2
0.5968147399901003
0.225
0.5946163649478823
0.25
0.5905873554875635
0.275
0.5946780720823348
0.30000000000000004
0.5929807164918675
0.32500000000000007
0.6017656836100477
0.35000000000000003
0.600477912669457
0.37500000000000006
0.5922373090771851
0.4
0.5945327696401969
0.42500000000000004
0.6040815392578779
0.45000000000000007
0.59587433067579
0.47500000000000003
0.5999744785931458


eta = 0.425

In [281]:
for learning_rate in [0.001, 0.01, 0.03, 0.05, 0.1, 0.5, 1]:
    print(learning_rate)
    print(my_score({**save, 'gamma':0.65, 'colsample_bytree':0.9, 'eta':0.425, 'learning_rate':learning_rate}))

0.001
0.585349715344713
0.01
0.590107254599438
0.03
0.5899367844804236
0.05
0.5948038612074255
0.1
0.5860239987910091
0.5
0.5975788976215111
1
0.591103488840597


In [286]:
for base_score in np.arange(0.1, 1, 0.05):
    print(base_score)
    print(my_score({**save, 'gamma':0.65, 'colsample_bytree':0.9, 'eta':0.425, 'base_score':base_score}))

0.1
0.5982274308923687
0.15000000000000002
0.5887978820086512
0.20000000000000004
0.5917770487616296
0.25000000000000006
0.5872864616494835
0.30000000000000004
0.5922342502547034
0.3500000000000001
0.6009972972649817
0.40000000000000013
0.5936507257438437
0.45000000000000007
0.5960528874444609
0.5000000000000001
0.5947762012120712
0.5500000000000002
0.5932241786134526
0.6000000000000002
0.5943127468789811
0.6500000000000001
0.5982613971772177
0.7000000000000002
0.5960427067704291
0.7500000000000002
0.5930253381033076
0.8000000000000002
0.5947875877127747
0.8500000000000002
0.5932193331695826
0.9000000000000002
0.5897381311381031
0.9500000000000003
0.5971725179743325


base_score = 0.35

In [290]:
print(my_score({**save, 'gamma':0.65, 'colsample_bytree':0.9, 'eta':0.425, 'base_score':0.35}))

0.6021009585726406


In [293]:
for alpha in [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    print(alpha)
    print(my_score({**save, 'gamma':0.65, 'colsample_bytree':0.9, 'eta':0.425, 'base_score':0.35, 'alpha':alpha}))

0
0.5965848203278703
0.0001
0.5953767601205436
0.001
0.5919406026771509
0.01
0.5941182154028479
0.1
0.6002872419602645
1
0.5966733914196068
10
0.5777130309875659
100
0.5518190831267822
1000
0.0


In [307]:
for alpha in [0.1, 0.2, 0.25, 0.3, 0.4]:
    print(alpha)
    print(my_score({**save, 'gamma':0.65, 'colsample_bytree':0.9, 'eta':0.425, 'base_score':0.35, 'alpha':alpha}))

0.1
0.6000175890412398
0.2
0.5986853562015959
0.25
0.5990771298738304
0.3
0.5954015743868051
0.4
0.5968610183098514


alpha = 0.1

In [312]:
for lambdaa in [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    print(lambdaa)
    print(my_score({**save, 'gamma':0.65, 'colsample_bytree':0.9, 'eta':0.425, 'base_score':0.35, 'alpha':0.1, 'lambda':lambdaa}))

0
0.5952088331961118
0.0001
0.5966248698798837
0.001
0.5996392178310755
0.01
0.5927957652265569
0.1
0.595232925797695
1
0.5924763795511434
10
0.5936095856045993
100
0.5755812081625248
1000
0.5585886837762817


In [313]:
save2 = {**save, 'gamma':0.65, 'colsample_bytree':0.9, 'eta':0.425, 'base_score':0.35, 'alpha':0.1}

In [320]:
for n in range(1, 51):
    save2['n_estimators'] = n
    print(n)
    print(my_score(save2))

1
0.4759478809533369
2
0.5448988927630598
3
0.5634093775106049
4
0.5710169189426454
5
0.5868772013211943
6
0.5905346177760462
7
0.5960908814639903
8
0.5916633241260465
9
0.5982952583737529
10
0.5966597435664788
11
0.5981280987707367
12
0.5972755912049739
13
0.6001879596800226
14
0.5950187675202325
15
0.5974126781553041
16
0.5976202667138892
17
0.5933591019447313
18
0.5995824105909375
19
0.6062460670873702
20
0.5927060754691271
21
0.6027938491245124
22
0.5963156651205075
23
0.6024526485237595
24
0.5944225009201873
25
0.5931443743168882
26
0.6031316151004398
27
0.5946987434956499
28
0.5874287428732047
29
0.5933916962722623
30
0.6033323205116895
31
0.5967281178967896
32
0.5906223963917066
33
0.5941898576903614
34
0.5936423340222967
35
0.6010459993724001
36
0.5938564717904069
37
0.5898570913085268
38
0.5923083743124526
39
0.596779343597101
40
0.5968385102399021
41
0.5926886232435817
42
0.5945896037322136
43
0.5949040780912525
44
0.5915202402092224
45
0.5955805566453611
46
0.597643622288704

In [321]:
save2['n_estimators'] = 30

In [367]:
for scale_pos_weight in [1.8, 1.9, 2, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 3, 3.5, 4, 4.5]:
    print(scale_pos_weight)
    print(my_score({**save2, 'scale_pos_weight':scale_pos_weight}))

1.8
0.6319648093841642
0.6444605358435915
0.6198224852071007
0.6228003060443765
0.6106442577030812
0.6259384788364628
1.9
0.615823235923022
0.6284074605451938
0.6364301389904902
0.6150506512301013
0.6397058823529412
0.6270834738083497
2
0.6479076479076479
0.6196581196581198
0.6218365871294288
0.613875262789068
0.6391901663051337
0.6284935567578797
2.1
0.6249120337790289
0.6284916201117318
0.6312769010043042
0.627177700348432
0.6329113924050633
0.6289539295297121
2.2
0.6322314049586777
0.6359832635983264
0.6206896551724138
0.6318644067796609
0.6240337315530569
0.6289604924124271
2.3
0.64
0.6337047353760447
0.6169014084507043
0.6356275303643724
0.6272022551092318
0.6306871858600707
2.4
0.6268456375838927
0.6284501061571125
0.6167582417582418
0.6416893732970027
0.6531678641410843
0.6333822445874668
2.5
0.6527027027027027
0.6361111111111112
0.6607629427792915
0.6293800539083558
0.600265604249668
0.6358444829502259
2.6
0.6413333333333334
0.6420765027322405
0.6188881446751507
0.6208754208754

In [365]:
test_data = pd.read_csv('test_groups.csv')
testgroups_titledata = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))
X_test = []
groups_test = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_test.append(sorted(all_dist, reverse=True)[0:25])
X_test = np.array(X_test)
groups_test = np.array(groups_test)
print(X_test.shape, groups_test.shape)

(16627, 25) (16627,)


In [372]:
clflast = clf = xgb.XGBClassifier(**{**save2, 'scale_pos_weight':2.8})
clflast.fit(X_train_sk, y_train)
y_pred = clflast.predict(X_train_sk)
print(f1_score(y_train, y_pred))
my_score({**save2, 'scale_pos_weight':3})

0.6914724576271185
0.6159844054580897
0.6356487549148099
0.6402116402116402
0.6096115865701118
0.6166883963494133


0.623628956700813

In [373]:
save2

{'n_estimators': 30,
 'max_depth': 10,
 'min_child_weight': 5,
 'subsample': 0.95,
 'eval_metric': 'auc',
 'gamma': 0.65,
 'colsample_bytree': 0.9,
 'eta': 0.425,
 'base_score': 0.35,
 'alpha': 0.1}

In [374]:
save2['scale_pos_weight'] = 2.5

In [382]:
for learning_rate in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.5, 1]:
    print(learning_rate)
    print(my_score({**save2, 'learning_rate':learning_rate}))

0.01
0.16094789825795464
0.02
0.6052012744850122
0.03
0.6264659523590559
0.04
0.6351681867433697
0.05
0.6359760884591168
0.06
0.6332906516247541
0.07
0.6359722470328618
0.08
0.6367327925676957
0.09
0.6375379740269671
0.1
0.6366136973104048
0.5
0.6280756054567357
1
0.624958051717016


learning_rate = 0.04 / 0.05

In [383]:
save2['learning_rate'] = 0.04

In [385]:
clflast = clf = xgb.XGBClassifier(**save2)
clflast.fit(X_train_sk, y_train)
y_pred = clflast.predict(X_train_sk)
print(f1_score(y_train, y_pred))
my_score(save2)

0.6749030311736819


0.6323959929996258

In [386]:
my_sk = StandardScaler()
X_test_sk = my_sk.fit_transform(X_test)
y_pred = clflast.predict(X_test_sk)
dataframe = pd.DataFrame(y_pred, columns=['target'])
sub = pd.concat([test_data['pair_id'], dataframe], axis=1)
sub.to_csv('submission.csv', index=False)

скор на тесте 0.616
пройдем еще раз все параметры уже с установленными другими

In [388]:
save2

{'n_estimators': 30,
 'max_depth': 10,
 'min_child_weight': 5,
 'subsample': 0.95,
 'eval_metric': 'auc',
 'gamma': 0.65,
 'colsample_bytree': 0.9,
 'eta': 0.425,
 'base_score': 0.35,
 'alpha': 0.1,
 'scale_pos_weight': 2.5,
 'learning_rate': 0.04}

In [391]:
for max_depth in range(2, 13):
    print(max_depth)
    save2['max_depth'] = max_depth
    print(my_score(save2))

2
0.6280954548132898
3
0.6319895078783991
4
0.6366382721572603
5
0.6406934216139623
6
0.6394022658712892
7
0.6378324841503965
8
0.635928048304787
9
0.6352935807493663
10
0.6373713320126098
11
0.6336106354652165
12
0.6292623246920246


In [392]:
save2['max_depth'] = 5

In [396]:
for min_child_weight in range(1, 11):
    print(min_child_weight)
    save2['min_child_weight'] = min_child_weight
    print(my_score(save2))

1
0.6391404405440231
2
0.6396743593084199
3
0.6411339868382814
4
0.6412754038547677
5
0.6401281774339583
6
0.638811485837314
7
0.6407446705248401
8
0.6407804968215779
9
0.6383357460225697
10
0.6377187865820049


In [397]:
save2['min_child_weight'] = 4

In [400]:
for subsample in np.arange(0.5, 1, 0.05):
    print(subsample)
    save2['subsample'] = subsample
    print(my_score(save2))

0.5
0.6401376172339949
0.55
0.6366918956421626
0.6000000000000001
0.641530726434379
0.6500000000000001
0.6380411826220774
0.7000000000000002
0.6404764290463937
0.7500000000000002
0.6406202600178657
0.8000000000000003
0.6403352829959021
0.8500000000000003
0.6409859058596624
0.9000000000000004
0.6397656084670852
0.9500000000000004
0.6417876947928756


subsample оставляем 0.95

In [401]:
for eval_metric in ['rmse', 'rmsle', 'mae', 'mphe', 'logloss', 'auc']:
    print(eval_metric)
    save2['eval_metric'] = eval_metric
    print(my_score(save2))

rmse
0.6397669555652171
rmsle
0.6398899976611221
mae
0.6406017983892416
mphe
0.6376506768098222
logloss
0.6415614484670054
auc
0.6419623751732461


In [402]:
save2

{'n_estimators': 30,
 'max_depth': 5,
 'min_child_weight': 4,
 'subsample': 0.9500000000000004,
 'eval_metric': 'auc',
 'gamma': 0.65,
 'colsample_bytree': 0.9,
 'eta': 0.425,
 'base_score': 0.35,
 'alpha': 0.1,
 'scale_pos_weight': 2.5,
 'learning_rate': 0.04}

In [409]:
for alpha in [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    print(alpha)
    save2['alpha'] = alpha
    print(my_score(save2))

0
0.6388130563663095
0.0001
0.6419539151910095
0.001
0.6395584918410722
0.01
0.638474368982049
0.1
0.6413876912934793
1
0.6394034254106802
10
0.6362580526793061
100
0.6307500369328467
1000
0.6277934536160775


In [410]:
save2['alpha'] = 0.0001

In [418]:
for lambdaa in [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    print(lambdaa)
    save2['lambda'] = lambdaa
    print(my_score(save2))

0
0.6397074099991956
0.0001
0.6415361632057555
0.001
0.640586472995702
0.01
0.6413835040123121
0.1
0.6418040169672561
1
0.64036201989142
10
0.6381721710391836
100
0.6329047483847076
1000
0.62851615606436


In [419]:
save2['lambda'] = 0.1

In [421]:
for n in range(1, 51):
    print(n)
    save2['n_estimators'] = n

1
0.0
2
0.0
3
0.0
4
0.0
5
0.0
6
0.0
7
0.0029673642761063007
8
0.37432351739092223
9
0.4757616072553608
10
0.49789369902586456
11
0.5219694630619199
12
0.5496746843505094
13
0.5664737306725853
14
0.5838153036614843
15
0.5988238562879096
16
0.6116920241822669
17
0.6225434530821997
18
0.6172897991467583
19
0.6311402901611269
20
0.628452070101583
21
0.6335697581548393
22
0.6332109119186737
23
0.6327547692163986
24
0.6331656815390666
25
0.6350535335286104
26
0.6413988335525446
27
0.6423456101924455
28
0.6416617526407642
29
0.6393368326857238
30
0.6415514535122663
31
0.6418651439730122
32
0.6420743768192354
33
0.6395921706164115
34
0.6427280770273494
35
0.6416659781673565
36
0.6426605235797337
37
0.6396645718397912
38
0.6402663062815326
39
0.6434967436874094
40
0.6427281197517334
41
0.6409407367063951
42
0.6395991989033613
43
0.6396628393513667
44
0.6412988403481314
45
0.6412987032830963
46
0.6383960272580532
47
0.6423743827991849
48
0.642708599408143
49
0.640442793935918
50
0.64142853727007

In [435]:
save2['n_estimators'] = 39

In [447]:
print(my_score(save2))

0.6424722635507732


In [449]:
for scale_pos_weight in [1.8, 1.9, 2, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 3, 3.5, 4, 4.5]:
    print(scale_pos_weight)
    save2['scale_pos_weight'] = scale_pos_weight
    print(my_score(save2))

1.8
0.6332295936358233
1.9
0.6332457618496857
2
0.6326026107601023
2.1
0.6358826392200543
2.2
0.6376350896023528
2.3
0.6420351228197193
2.4
0.6421980915384023
2.5
0.6410553833767707
2.6
0.640760231486671
3
0.6336450939429882
3.5
0.6320398539755293
4
0.6306409082095672
4.5
0.6284786678613271


In [450]:
save2['scale_pos_weight'] = 2.4

In [453]:
clflast = clf = xgb.XGBClassifier(**save2)
clflast.fit(X_train_sk, y_train)
y_pred = clflast.predict(X_train_sk)
print(f1_score(y_train, y_pred))
my_score(save2)

0.6498472978356128


0.6437404255386436

In [454]:
my_sk = StandardScaler()
X_test_sk = my_sk.fit_transform(X_test)
y_pred = clflast.predict(X_test_sk)
dataframe = pd.DataFrame(y_pred, columns=['target'])
sub = pd.concat([test_data['pair_id'], dataframe], axis=1)
sub.to_csv('submission.csv', index=False)

скор 0.63457 УРА!